# Monte-Carlo Simulations

We'll be using the htex configuration for Parsl. Read more [here.]( https://github.com/Parsl/parsl/blob/master/parsl/configs/htex_local.py)

In [2]:
# External Libraries
import numpy as np

# Importing Parsl and the htex configuration
import parsl
from parsl.app.app import python_app, bash_app
from parsl.providers import LocalProvider
from parsl.channels import LocalChannel

from parsl.config import Config
from parsl.executors import HighThroughputExecutor

config = Config(
    executors=[
        HighThroughputExecutor(
            label="htex_local",
            cores_per_worker=1,
            provider=LocalProvider(
                channel=LocalChannel(),
                init_blocks=1,
                max_blocks=1,
            ),
        )
    ],
)

parsl.load(config)

Process Process-1:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/parsl/executors/high_throughput/interchange.py", line 539, in starter
    ic.start()
  File "/usr/local/lib/python3.7/site-packages/parsl/executors/high_throughput/interchange.py", line 348, in start
    self.socks = dict(poller.poll(timeout=poll_period))
  File "/usr/local/lib/python3.7/site-packages/zmq/sugar/poll.py", line 99, in poll
    return zmq_poll(self.sockets, timeout=timeout)
  File "zmq/backend/cython/_poll.pyx", line 123, in zmq.backend.cython._poll.zmq_poll
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.che

Let us work with a simple trading function that estimates the change in a stock if there is a volatility of 0.5% every day.

In [6]:
# Building a python app that generates a random percentage between 0.5 and -0.5, and then adjusts the stock price.

@python_app
def stock_price(price):
    import random
    percentage_change = random.random()-0.5
    
    return price*percentage_change/100

In [7]:
# Evaluating the final prices

final_prices = []

for _ in range(1000): ## A 1000 simulations
    
    original_price = 15
    for i in range(100): # For each simulation, we simulate the stock price over 100 days
        original_price = stock_price(original_price)
    
    final_prices.append(original_price)

In [ ]:
final_prices = [i.result() for i in final_prices]

In [ ]:
print('Mean Stock Price: ', np.mean(final_prices))
print('95% Confidence Interval of Stock Price: ', (np.percentile(final_prices,2.5),
                                                   np.percentile(final_prices,97.5))